In [46]:
# convert 3d data to labeled image data
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import time
import datetime
from pathlib import Path
import glob
import json
import math

tf.random.set_seed(1234)

with open("labaled_data.json", "r") as outfile:
    pointcloud_data = json.load(outfile)
        
with open("our_labaled_data.json", "r") as outfile:
    our_data = json.load(outfile)
    
pointcloud_data.update(our_data)

classes = set()
for i in pointcloud_data:
    classes.add(pointcloud_data.get(i)[1])
classes = list(classes)

# mp_face_detection = mp.solutions.face_detection
# mp_drawing = mp.solutions.drawing_utils

images_path = list(pointcloud_data.keys())
# display(images_path[0])
image_path = images_path[650]

image = cv2.imread(image_path)
dimensions = image.shape

# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

height = image.shape[0]
width = image.shape[1]
channels = image.shape[2]

print('Image Dimension    : ',dimensions)
print('Image Height       : ',height)
print('Image Width        : ',width)
print('Number of Channels : ',channels)

EXTRA_PADDING=0.02
points = np.array(pointcloud_data.get(image_path)[0])
# min_x = min(points[:, 0])
# max_x = max(points[:, 0])
# min_y = min(points[:, 1])
# max_y = max(points[:, 1])
min_x = (min(points[:, 0])-EXTRA_PADDING) if (min(points[:, 0])-EXTRA_PADDING) > 0 else 0
max_x = (max(points[:, 0])+EXTRA_PADDING) if (min(points[:, 0])+EXTRA_PADDING) < 1 else 1
min_y = (min(points[:, 1])-EXTRA_PADDING) if (min(points[:, 0])-EXTRA_PADDING) > 0 else 0
max_y = (max(points[:, 1])+EXTRA_PADDING) if (min(points[:, 0])+EXTRA_PADDING) < 1 else 1

# if (max_x-min_x) < (max_y-min_y):
#     max_x=((max_x+min_x)/2)+((max_y-min_y)/2)
#     min_x=((max_x+min_x)/2)-((max_y-min_y)/2)
#     if min_x < 0 :
#         min_x = 0
#     if max_x > 1:
#         max_x = 1
# elif (max_x-min_x) > (max_y-min_y):
#     max_y=((max_y+min_y)/2)+((max_x-min_x)/2)
#     min_y=((max_y+min_y)/2)-((max_x-min_x)/2)
#     if min_y < 0 :
#         min_y = 0
#     if max_y > 1:
#         max_y = 1
        
display(min_x,max_x,min_y,max_y)

#coordinate are inverted here
face_Coor_min_x = int((min_x)*width)
face_Coor_max_x = int((max_x)*width)
face_Coor_min_y = int((min_y)*height)
face_Coor_max_y = int((max_y)*height)

#square up the face crop
if (face_Coor_max_x-face_Coor_min_x) < (face_Coor_max_y-face_Coor_min_y):
    face_Coor_max_x=math.ceil((face_Coor_max_x+face_Coor_min_x)/2)+math.ceil((face_Coor_max_y-face_Coor_min_y)/2)
    face_Coor_min_x=math.floor((face_Coor_max_x+face_Coor_min_x)/2)-math.ceil((face_Coor_max_y-face_Coor_min_y)/2)
    if face_Coor_min_x < 0 :
        face_Coor_min_x = 0
    if face_Coor_max_x > width:
        face_Coor_max_x = width
elif (face_Coor_max_x-face_Coor_min_x) > (face_Coor_max_y-face_Coor_min_y):
    face_Coor_max_y=math.ceil((face_Coor_max_y+face_Coor_min_y)/2)+math.ceil((face_Coor_max_x-face_Coor_min_x)/2)
    face_Coor_min_y=math.floor((face_Coor_max_y+face_Coor_min_y)/2)-math.ceil((face_Coor_max_x-face_Coor_min_x)/2)
    if face_Coor_min_y < 0 :
        face_Coor_min_y = 0
    if face_Coor_max_y > height:
        face_Coor_max_y = height

#make face crops completely square
count =0
while (face_Coor_max_x-face_Coor_min_x) != (face_Coor_max_y-face_Coor_min_y):
    if (face_Coor_max_x-face_Coor_min_x) < (face_Coor_max_y-face_Coor_min_y):
        if count%2==0:
            face_Coor_max_x+=1
        elif count%2==1:
            face_Coor_min_x-=1
    elif (face_Coor_max_x-face_Coor_min_x) > (face_Coor_max_y-face_Coor_min_y):
        if count%2==0:
            face_Coor_max_y+=1
        elif count%2==1:
            face_Coor_min_y-=1
    count+=1

    

display(face_Coor_min_x, face_Coor_max_x, face_Coor_min_y, face_Coor_max_y)
# min_z = (min(points[:, 2])-0.05) if (min(points[:, 0])-0.05) > 0 else 0
# max_z = (max(points[:, 2])+0.05) if (min(points[:, 0])-0.05) < 1 else 1
# display(min_x, max_x,min_y, max_y, min_z, max_z)
# print(int(min_x*width))
face = image[face_Coor_min_y:face_Coor_max_y, face_Coor_min_x:face_Coor_max_x]
display(face.shape)
# scale_percent = 150 # percent of original size
# new_width = int(face.shape[1] * scale_percent / 100)
# new_height = int(face.shape[0] * scale_percent / 100)
dim = (255, 255)

resized = cv2.resize(face, dim, interpolation = cv2.INTER_AREA)

display(resized.shape)
display(pointcloud_data.get(image_path)[1])
# display(resized[0][0])
cv2.imshow("Cropped Face", face)
cv2.imshow("Cropped and Resized Face", resized)
cv2.imshow("original", image)
cv2.waitKey(0)

#     plt.imshow(annotated_image, cmap='gray')


Image Dimension    :  (480, 640, 3)
Image Height       :  480
Image Width        :  640
Number of Channels :  3


0.3924578237533569

0.677574474811554

0.31618825674057005

0.7457978320121765

...

357

(206, 206, 3)

(255, 255, 3)

'open_mouth'

-1

In [ ]:
0.22978347539901733

0.20633304119110107

0.7202502489089966

0.784380316734314